In [1]:
import pandas as pd
import numpy as np
import csv
from rdkit import Chem

# Expression

In [2]:
expression_df=pd.read_csv("./data/RawFile/CCLE_expression.csv",index_col=0)

In [3]:
entrez_dict = {}
drop_list = []
for col in expression_df.columns:
    split = col.split(" (")
    entrez = split[1][:-1]
    symbol = split[0]
    if entrez not in list(entrez_dict.keys()):
        entrez_dict[entrez] = (symbol,col)
    else:
        drop_list.append(col) 

new_expression_df = expression_df.drop(columns=drop_list)

In [5]:
new_expression_df.to_csv("./data/ProcessedFile/22Q1expressions.csv")

# Drug

In [ ]:
drug_info = pd.read_csv("./data/RawFile/secondary-screen-replicate-collapsed-treatment-info.csv")

In [ ]:
def canonicalize_smiles(smi):
    if not smi:
        return ""
    try:
        mol = Chem.MolFromSmiles(smi)
        if mol is None:
            return ""
        return Chem.MolToSmiles(mol)
    except:
        return ""

In [ ]:
keep_drug_info = drug_info[['name','smiles']].drop_duplicates()
keep_drug_info['right_smiles'] = [smi.split(",")[0].strip() for smi in keep_drug_info['smiles']]
keep_drug_info['canonical_smiles'] = [canonicalize_smiles(smi) for smi in keep_drug_info['right_smiles']]
keep_drug_info = keep_drug_info[keep_drug_info['canonical_smiles']!=""]

In [ ]:
keep_drug_info.to_csv("./data/ProcessedFile/sec_auc_drug.csv",index=False)

# Response

In [ ]:
response = pd.read_csv("./data/RawFile/secondary-screen-dose-response-curve-parameters.csv",low_memory=False)

In [ ]:
response = response.dropna(subset=['depmap_id'])
response =response [response['passed_str_profiling'] == True]

keep_response = response[['name','depmap_id','auc']].drop_duplicates(subset=['name','depmap_id']).copy(deep=True)
keep_response = keep_response[(keep_response['depmap_id'].isin(new_expression_df.index)) & (keep_response['name'].isin(keep_drug_info['name']))].copy(deep=True)

In [ ]:
keep_response.to_csv("./data/ProcessedFile/Prism_secondary_auc.csv",index=False)